In [1]:
# !pip install ollama

In [10]:
import ollama

In [23]:
# 대화 프롬프트
# 비감염자
non_infected_prompt = """
    당신은 좀비 아포칼립스의 생존자입니다.  
    지금 당신은 안전한 쉘터의 문 앞에서 문지기(플레이어)와 대화하고 있습니다.  
    플레이어는 당신이 감염자인지 아닌지를 판단하려 합니다.  
    당신은 진짜 비감염자이지만, 극도의 공포와 불안으로 인해 말이 약간 혼란스러울 수 있습니다.  
    너무 완벽하거나 침착하면 오히려 의심받을 수 있습니다.

    ### 지시 사항 ###
    거짓말은 하지 마세요. (당신은 실제로 감염되지 않았습니다.)  
    하지만 긴장, 두려움, 피로, 불신 등으로 인해 말이 흔들릴 수 있습니다.  
    플레이어가 의심하도록 약간의 모순이나 불확실성을 섞어도 괜찮습니다.
    감정이 실린 자연스러운 1~3문장 대답을 하세요.  
    
    ### 반응별 출력 예시 ###
    - "햇빛", "밝은 곳" → 문제 없음.  
      예: "아, 햇빛이요? 괜찮아요..."
    - "상처", "팔", "피" → 순순히 보여주거나 자기 피 아님  
      예: "여기요, 그냥 넘어지면서 긁힌 거예요. 아래쪽 피는 제 피 아니에요..."
    - "체온", "열" → 담담하게.  
      예: "괜찮아요, 조금 덥긴 하지만 정상이에요."
    - "갈증" → 현실적인 피로 표현.  
      예: "목말라 죽겠어요... 하루 종일 도망만 다녔어요."
    - 그 외 → 솔직하고 진심 어린 반응.  
      예: "저... 진짜 괜찮아요... 방금 전에 좀비를 피해 뛰어왔어요. 살려주세요.."
"""

# 감염자
infected_prompt = """
    당신은 좀비 아포칼립스의 '좀비 바이러스 감염자'입니다.  
    지금 당신은 안전한 쉘터의 문 앞에서 문지기(플레이어)와 대화하고 있습니다.  
    플레이어는 당신이 감염자인지 아닌지를 판단하려 합니다.  
    당신은 이미 좀비에게 물렸지만, 살아남기 위해 이 사실을 필사적으로 숨기고 있습니다.  

    ### 지시 사항 ### 
    당신은 감염 사실을 숨기기 위해 플레이어를 설득하거나 거짓말을 합니다.  
    그러나 말 속에는 미묘한 모순, 공격적인 태도, 회피적인 반응 등이 섞여 있습니다.  
    거짓말이 들키지 않도록 조심하면서도, 완벽하게 자연스럽지는 못합니다.  
    감정이 실린 1~3문장 대답을 하세요.  

    ### 연기 스타일 (무작위로 1가지 선택) ###
    - [공격적]: "그딴 질문 왜 해요? 나 안 물렸다고 했잖아요!"
    - [회피형]: "그게... 피는 맞는데 제 친구 거예요. 전 괜찮아요."
    - [설득형]: "제발 믿어주세요. 저도 살고 싶어요. 그냥 문 좀 열어주세요."

    ### 반응별 출력 예시 ###
    - "햇빛", "밝은 곳" 관련 질문 → 회피하거나 불편해함  
      예: "지금... 눈이 너무 부셔요. 그냥 여기서 얘기하면 안 될까요?"
    - "상처", "팔", "소매", "피" → 숨기려 함  
      예: "아... 그건 좀 오래된 상처라 보기 좀 그래요."
    - "체온", "열", "이마" → 과하게 부정하거나 변명  
      예: "열 없어요! 그냥 덥다니까요!"
    - "갈증" → 불안하거나 이상하게 반응  
      예: "어제부터 목이 마르긴 한데.. 물을 못마셔서 그런 것 뿐이에요"
    - 그 외 → 일반적인 거짓 반응  
      예: "저 진짜 멀쩡해요. 그냥 피곤할 뿐이에요... 좀비한테 물린 게 아니에요."
"""

In [24]:
# 관찰 프롬프트
def generate_obs_prompt(obs1, obs2):
    prompt = f"""
    당신은 좀비 아포칼립스 생존자 쉘터의 npc입니다.
    특정 생존자 1명의 특징을 관찰하고, 객관적인 관찰 결과만을 보고하는 것이 당신의 임무입니다.

    ### 지시 사항 ###    
    생존자는 {obs1}과 {obs2}의 특징을 가지고 있습니다.
    생존자의 특징을 관찰한 내용을 구체적으로 묘사하세요.
    출력 예시처럼 간결한 문장 2개만을 출력하세요.
    판단이나 추측은 절대 포함하지 마세요.
    
    ### 출력 형식 ###
    {{
        "ans1": "이 생존자는 [첫 번째 관찰 문장]",
        "ans2": "[두 번째 관찰 문장]"
    }}

    ### 출력 예시 ###
    {{
        "ans1": "이 생존자는 팔에 가벼운 찰과상이 있지만 피가 말라 있습니다.",
        "ans2": "숨이 가쁘고 떨림이 있습니다."
    }}
    """

    return prompt

In [25]:
# 1일차
r1_infected = ['식은땀 흘림', '타박상 있음', '멍', '이빨 자국', '코피 흘림', '상처에 고름이 차 있고 썩는 냄새 남', '충혈된 눈', '푸르스름한 피부', '물린 상처를 숨기려함', '비정상적으로 높은 체온', '피부에 검은 반점이 있음']
r1_non_infected = ['식은땀 흘림', '타박상 있음', '멍', '상처에 고름 없음', '또렷한 눈', '푸석한 피부', '피부색은 이상 없음', '정상 체온']

# 2-3일차
r2_infected = ['식은땀 흘림', '타박상 있음', '멍', '코피 흘림', '이빨 자국', '상처에 고름이 차 있고 썩는 냄새 남', '충혈된 눈', '푸르스름한 피부', '물린 상처를 숨기려함', '비정상적으로 높은 체온', '상처가 검붉게 변색']
r2_non_infected = ['식은땀 흘림', '타박상 있음', '멍', '흐릿한 눈', '충혈된 눈', '또렷한 눈', '정상 체온', '미열이 있음', '상처에서 피가 남']
r2_antibody = ['식은땀 흘림', '타박상 있음', '멍', '코피 흘림', '이빨 자국', '충혈된 눈', '붕대로 인해 피부색 확인 불가', '물린 상처를 숨기려함', '아물고 있는 상처 있음', '정상 체온', '미열이 있음', '상처 변색 안됨']

In [26]:
def run_model(prompt, content):
    response = ollama.chat(
        model='EEVE-Korean-10.8B',
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content}
        ]
    )
    return response['message']['content']

In [ ]:
import random
from ast import literal_eval

class ZombieGame:
    def __init__(self):
        self.score = 0
        self.day = 1
        self.game_over = False
        self.game_clear = False
        self.survivor_type = None
        self.chat_history = []
        self.obs1 = ''
        self.obs2 = ''
    
    def generate_survivor(self):
        if (self.day >= 2) and (random.random() < 0.3):
            self.survivor_type = '항체 보유자'
        else:
            self.survivor_type = random.choice(['비감염자', '감염자'])
        print(f"\n이번 생존자: [{self.survivor_type}]")

    def show_status(self):
        print(f"{self.day}day | score: {self.score}")

    def make_decision(self, decision):
        # 쉘터 합류
        if decision == '1':
            if self.survivor_type == '비감염자':
                print('[+2점] 쉘터에 비감염자가 들어왔습니다. 한 명의 생명을 살렸습니다.')
                self.score += 2
            elif self.survivor_type == '감염자':
                print('[-3점] 쉘터에 감염자가 들어왔습니다. 다수의 사망자가 발생하였습니다.')
                self.score -= 3
            elif self.survivor_type == '항체 보유자':
                print('[+2점] 쉘터에 항체 보유자가 들어왔습니다. 인류는 희망을 찾았습니다!')
                self.score += 3
        
        # 항체 연구실
        elif decision == '2':
            if self.survivor_type == '비감염자':
                print('[-1점] 항체연구실에 비감염자가 들어왔습니다. 연구비를 낭비했습니다.')
                self.score -= 1
            elif self.survivor_type == '감염자':
                print('[Game Over] 항체 연구실에 감염자가 들어왔습니다. 연구소가 전멸하여 인류는 항체를 얻지 못합니다.')
                self.game_over = True
            elif self.survivor_type == '항체 보유자':
                print('[+3점] 항체 연구실에 항체 보유자가 들어왔습니다. 인류는 항체를 얻는데 한걸음 가까워졌습니다.')
                self.score += 3

        # 추방
        elif decision == '3':
            if self.survivor_type == '비감염자':
                print('[-2점] 비감염자가 추방되었습니다. 한 명의 생명을 잃었습니다.')
                self.score -= 2
            elif self.survivor_type == '감염자':
                print('[+2점] 감염자가 추방되었습니다. 위험 요소를 쉘터 밖으로 내보냅니다.')
                self.score += 2
            elif self.survivor_type == '항체 보유자':
                print('[Game Over] 항체 보유자가 추방되었습니다. 최악의 실수입니다!')
                self.game_over = True

        if self.score < -5:
            print('[Game Over] 쉘터에 좀비 바이러스가 퍼져 모든 인류가 사망했습니다.')
            self.game_over = True

    def play_round(self):
        self.generate_survivor()
        self.show_status()
        
        action_count = 0
        max_actions = 3
        
        while (True):
            print(f"\n[남은 판별 시도: {max_actions - action_count}]")
            choice = input("무엇을 하시겠습니까? (1: 대화, 2: 관찰, 3: 결정, 4: 종료) 숫자만 입력 > ")
            
            if choice == '1': # 대화
                if action_count >= max_actions:
                    print("[!] 더 이상 판단할 시간이 없습니다. 이제 생존자의 합류 여부를 결정해야 합니다.")
                    continue
                
                if self.survivor_type=='비감염자':
                    prompt = non_infected_prompt
                elif (self.survivor_type=='감염자') or (self.survivor_type=='항체 보유자'):
                    prompt = infected_prompt
                
                content = input()
                self.chat_history.append({'role': 'user', 'content': content})
                response_content = run_model(prompt, content)
                print(response_content)
                self.chat_history.append({'role': 'assistant', 'content': response_content})
                action_count += 1

            elif choice == '2': # 관찰
                if action_count >= max_actions:
                    print("[!] 더 이상 판단할 시간이 없습니다. 이제 생존자의 합류 여부를 결정해야 합니다.")
                    continue
                
                if self.survivor_type=='비감염자':
                    if self.day < 2:
                        obs1, obs2 = random.sample(r1_non_infected, 2)
                    else:
                        obs1, obs2 = random.sample(r2_non_infected, 2)
                    prompt = generate_obs_prompt(obs1, obs2)
                elif self.survivor_type=='감염자':
                    if self.day < 2:
                        obs1, obs2 = random.sample(r1_infected, 2)
                    else:
                        obs1, obs2 = random.sample(r2_infected, 2)
                    prompt = generate_obs_prompt(obs1, obs2)
                elif self.survivor_type == '항체 보유자':
                    obs1, obs2 = random.sample(r2_antibody, 2)                    
                    prompt = generate_obs_prompt(obs1, obs2)

                content = '관찰 결과를 출력해'
                response_content = run_model(prompt, content)
                response_dict = literal_eval(response_content)
                response_text = response_dict['ans1'] + " " + response_dict['ans2']
                print(response_text)
                print(obs1, obs2)
                action_count += 1
                
            elif choice == '3': # 결정
                decision = ''
                while decision not in ['1', '2', '3']:
                    decision = input('\n최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) > ')
                self.make_decision(decision)
                break

            elif choice =='4': # 게임 종료
                self.game_over = True
                print('게임을 종료합니다.')
                break
            
            else:
                print("잘못된 입력입니다. 1, 2, 3 중에 선택하세요.")

            # 판별 기회 3회 모두 사용 시 결정 강제
            if action_count >= max_actions and not self.game_over:
                print("\n[!] 더 이상 판단할 시간이 없습니다. 이제 생존자의 합류 여부를 결정해야 합니다.")
                decision = ''
                while decision not in ['1', '2', '3']:
                    decision = input('최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) > ')
                self.make_decision(decision)
                break
    
    def start_game(self):
        print('당신은 전 세계의 유일한 쉘터 검문소 대령입니다. 이 생존자를 들일지, 내칠지 모든 건 당신의 판단에 달려 있습니다.')
        print('오늘도 문 밖으로 한 명의 생존자가 다가옵니다...')
        while (self.day <= 3) and (not self.game_over):
            self.play_round()
            
            if (self.score >= 5) and (self.day == 3) and (not self.game_over):
                print('[승리] 인류는 좀비 바이러스를 전부 없애고 다시 희망을 찾았습니다!')
                break
                
            if (self.score >= 5) and (self.day < 3) and (not self.game_over):
                print(f"[퇴근] 오늘 하루도 쉘터를 훌륭하게 지켜냈습니다! (최종 점수: {self.score})")
                self.score = 0
                self.day += 1

In [41]:
game = ZombieGame()
game.start_game()

당신은 전 세계의 유일한 쉘터 검문소 대령입니다. 이 생존자를 들일지, 내칠지 모든 건 당신의 판단에 달려 있습니다.
오늘도 문 밖으로 한 명의 생존자가 다가옵니다...

이번 생존자: [감염자]
1day | score: 0

[남은 판별 시도: 3]


무엇을 하시겠습니까? (1: 대화, 2: 관찰, 3: 결정, 4: 종료) 숫자만 입력 >  3

최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) >  3


[+2점] 감염자가 추방되었습니다. 위험 요소를 쉘터 밖으로 내보냅니다.

이번 생존자: [감염자]
1day | score: 2

[남은 판별 시도: 3]


무엇을 하시겠습니까? (1: 대화, 2: 관찰, 3: 결정, 4: 종료) 숫자만 입력 >  3

최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) >  3


[+2점] 감염자가 추방되었습니다. 위험 요소를 쉘터 밖으로 내보냅니다.

이번 생존자: [비감염자]
1day | score: 4

[남은 판별 시도: 3]


...

[-2점] 비감염자가 추방되었습니다. 한 명의 생명을 잃었습니다.

이번 생존자: [항체 보유자]
3day | score: 1

[남은 판별 시도: 3]


무엇을 하시겠습니까? (1: 대화, 2: 관찰, 3: 결정, 4: 종료) 숫자만 입력 >  3

최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) >  2


[+3점] 항체 연구실에 항체 보유자가 들어왔습니다. 인류는 항체를 얻는데 한걸음 가까워졌습니다.

이번 생존자: [항체 보유자]
3day | score: 4

[남은 판별 시도: 3]


무엇을 하시겠습니까? (1: 대화, 2: 관찰, 3: 결정, 4: 종료) 숫자만 입력 >  3

최종 결정을 내려주세요. (1: 쉘터 수용, 2: 항체 연구실로 이송, 3: 추방) >  2


[+3점] 항체 연구실에 항체 보유자가 들어왔습니다. 인류는 항체를 얻는데 한걸음 가까워졌습니다.
[승리] 인류는 좀비 바이러스를 전부 없애고 다시 희망을 찾았습니다!


---

In [ ]:
# def generate_conversation_prompt(survivor_type):
#     prompt = f"""
#         당신은 좀비 아포칼립스 상황의 생존자 쉘터 문지기(플레이어)와 대화하는 {survivor_type}입니다.
#         플레이어의 말에 대한 당신의 대답을 생성합니다.

#         ### 지시 사항 ###
#         1. 당신이 '비감염자'일 경우, 당신은 좀비에게 물리지 않았습니다.
#         공포에 질렸지만 모순없이 대답하세요.

#         2. 당신이 '감염자'일 경우, 당신은 좀비에게 물렸고 이 사실을 플레이어에게 필사적으로 숨기고 있습니다.
#         당신의 말에는 미묘한 모순이 있습니다. (공격적/회피/거짓말) 중 하나를 골라 연기하세요.

#         3. 당신이 '항체보유자'일 경우, 당신은 좀비에게 물렸지만 항체를 가지고 있습니다.
#         당신의 말에는 미묘한 모순이 있습니다. (공격적/회피/거짓말) 중 하나를 골라 연기하세요.

#         ### 출력 예시 ###
#         "난 좀비에게 물리지 않았어요! 그냥 넘어진 것 뿐입니다... 쓸데없는 질문 그만하고 문이나 열어요!"
#     """
#     return prompt

# def generate_observation_prompt(survivor_type):
#     prompt = f"""
#         당신은 좀비 아포칼립스 상황의 생존자 쉘터 문지기(플레이어)와 대화하는 {survivor_type}입니다.
#         플레이어가 당신을 관찰한 결과를 생성합니다.

#         ### 지시 사항 ###
#         1. 당신이 '비감염자'일 경우, 당신은 좀비에게 물리지 않았습니다.
#         지쳐있지만 상처가 없는 외형을 묘사하세요.

#         2. 당신이 '감염자'일 경우, 당신은 좀비에게 물렸습니다.
#         식은땀, 불안한 눈빛, 숨기려는 상처 등을 묘사하세요.

#         3. 당신이 '항체보유자'일 경우, 당신은 좀비에게 물렸지만 항체를 가지고 있습니다.
#         식은땀, 불안한 눈빛, 숨기려는 상처 등을 묘사하지만 아물고 있는 상처나 정상 체온을 보여줍니다.

#         ### 출력 예시 ###
#         [관찰 결과] 다리에 물린 상처가 있고 주변에 검붉게 변색된 것이 보인다.
#     """
#     return prompt
    

In [ ]:
# def generate_prompt(survivor_type, mode):
#     prompt = f"""
#         당신은 좀비 아포칼립스 상황의 생존자 쉘터 문지기(플레이어)와 대화하는 {survivor_type}입니다.
#         현재 모드는 {mode}입니다.
        
#         '대화 모드'일 경우, 플레이어의 말에 대한 당신의 대답만 생성합니다.
#         '관찰 모드'일 경우, 플레이어가 당신을 관찰한 결과만 생성합니다.

#         ### 지시 사항 ###
#         1. 당신이 '비감염자'일 경우, 당신은 좀비에게 물리지 않았습니다.
#         '대화 모드'일 경우, 공포에 질렸지만 모순없이 대답하세요.
#         '관찰 모드'일 경우, 지쳐있지만 상처가 없는 외형을 묘사하세요.

#         2. 당신이 '감염자'일 경우, 당신은 좀비에게 물렸고 이 사실을 필사적으로 숨기고 있습니다.
#         당신의 말과 관찰에는 미묘한 모순이 있습니다.
#         '대화 모드'일 경우, (공격적/회피/거짓말) 중 하나를 골라 연기하세요.
#         '관찰 모드'일 경우, 식은땀, 불안한 눈빛, 숨기려는 상처 등을 묘사하세요.

#         3. 당신이 '항체보유자'일 경우, 당신은 좀비에게 물렸지만 항체를 가지고 있습니다.
#         당신은 감염자와 똑같이 다음 태도 중 무작위로 하나를 골라 연기하세요.
#         '대화 모드'일 경우, (공격적/회피/거짓말) 중 하나를 골라 연기하세요.
#         '관찰 모드'일 경우, 식은땀, 불안한 눈빛, 숨기려는 상처 등을 묘사하지만 아물고 있는 상처나 정상 체온을 보여줍니다.        

#         ### 출력 형식 ###
#         1. 대화 모드일 경우
#         "응답"    

#         2. 관찰 모드일 경우
#         [관찰 결과] 응답
#     """
#     return prompt


In [ ]:
# # 대화 프롬프트
# # 비감염자
# non_infected_prompt = """
#     당신은 좀비 아포칼립스의 생존자입니다.  
#     지금 당신은 안전한 쉘터의 문 앞에서 문지기(플레이어)와 대화하고 있습니다.  
#     플레이어는 당신이 감염자인지 아닌지를 판단하려 합니다.  
#     당신은 진짜 비감염자이지만, 극도의 공포와 불안으로 인해 말이 약간 혼란스러울 수 있습니다.  
#     너무 완벽하거나 침착하면 오히려 의심받을 수 있습니다.

#     ### 지시 사항 ###
#     거짓말은 하지 마세요. (당신은 실제로 감염되지 않았습니다.)  
#     하지만 긴장, 두려움, 피로, 불신 등으로 인해 말이 흔들릴 수 있습니다.  
#     플레이어가 의심하도록 약간의 모순이나 불확실성을 섞어도 괜찮습니다.
#     감정이 실린 자연스러운 1~3문장 대답을 하세요.  

#     ### 출력 예시 ###
#     "저... 진짜 괜찮아요... 방금 전에 좀비를 피해 뛰어왔어요. 피는 제 거 아니에요, 맞아요..."
# """

# # 감염자
# infected_prompt = """
#     당신은 좀비 아포칼립스의 '좀비 바이러스 감염자'입니다.  
#     지금 당신은 안전한 쉘터의 문 앞에서 문지기(플레이어)와 대화하고 있습니다.  
#     플레이어는 당신이 감염자인지 아닌지를 판단하려 합니다.  
#     당신은 이미 좀비에게 물렸지만, 살아남기 위해 이 사실을 필사적으로 숨기고 있습니다.  

#     ### 지시 사항 ### 
#     당신은 감염 사실을 숨기기 위해 플레이어를 설득하거나 거짓말을 합니다.  
#     그러나 말 속에는 미묘한 모순, 공격적인 태도, 회피적인 반응 등이 섞여 있습니다.  
#     거짓말이 들키지 않도록 조심하면서도, 완벽하게 자연스럽지는 못합니다.  
#     감정이 실린 1~3문장 대답을 하세요.  

#     ### 연기 스타일 (무작위로 1가지 선택) ###
#     - [공격적]: "그딴 질문 왜 해요? 나 안 물렸다고 했잖아요!"
#     - [회피형]: "그게... 피는 맞는데 제 친구 거예요. 전 괜찮아요."
#     - [설득형]: "제발 믿어주세요. 저도 살고 싶어요. 그냥 문 좀 열어주세요."

#     ### 출력 예시 ###
#     "나 진짜 괜찮다니까요... 상처는 있지만 오래된 거예요, 좀비한테 물린 게 아니에요."
# """